<a href="https://colab.research.google.com/github/yukiharada1228/Optuna_Samples/blob/main/filters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Optunaをインストール
!pip install optuna

In [ ]:
BATCH_SIZE = 128
H = 28
W = 28
KERNEL_SIZE = 3

EPOCH = 20
N_TRIALS = 100

In [ ]:
# データセットを準備
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision import transforms


transform = transforms.Compose([transforms.ToTensor(),])

train_set = MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)

test_set = MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# モデルを定義
import torch
import torch.nn as nn
import torch.nn.functional as F


class CNN(nn.Module):
    def __init__(self, num_filters):
        super(CNN, self).__init__()
        self.act = F.relu
        
        self.convs = nn.ModuleList([nn.Conv2d(in_channels=1, out_channels=num_filters[0], kernel_size=KERNEL_SIZE)])
        self.out_height = H - KERNEL_SIZE + 1
        self.out_width = W - KERNEL_SIZE + 1

        self.out_height //= 2
        self.out_width //= 2

        for i in range(1, 3):
            self.convs.append(nn.Conv2d(in_channels=num_filters[i-1], out_channels=num_filters[i], kernel_size=KERNEL_SIZE))
            self.out_height -= KERNEL_SIZE - 1
            self.out_width -= KERNEL_SIZE - 1

            self.out_height //= 2
            self.out_width //= 2
        
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.out_feature = self.out_height * self.out_width * num_filters[-1]
        self.fc1 = nn.Linear(in_features=self.out_feature, out_features=1024)
        self.fc2 = nn.Linear(in_features=1024, out_features=10)
    
    def forward(self, x):
        for conv in self.convs:
            x = self.pool(self.act(conv(x)))
        x = x.view(-1, self.out_feature)
        x = self.act(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# 訓練
def train(model, device, train_loader, optimizer):
  model.train()
  for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
# 検証
def test(model, device, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    return 1 - correct / len(test_loader.dataset)

In [ ]:
# 目的関数
def objective(trial):
    device = "cuda" if torch.cuda.is_available() else "cpu" 
    # フィルタ数の候補
    num_filters = [int(trial.suggest_int("num_filter_"+str(i), 256, 1024, 96)) for i in range(3)]  
    model = CNN(num_filters).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)    
    print('device:', device)
    print('num_filters:', num_filters)
    print('optimizer:', optimizer)  
    for step in range(EPOCH):
        train(model, device, train_loader, optimizer)
        error_rate = test(model, device, test_loader)
        print({'step': step, 'acc': str(1-error_rate)})
    return error_rate

In [ ]:
# パラメータチューニング実行
import optuna
optuna.logging.enable_default_handler()


study = optuna.create_study()
study.optimize(objective, n_trials=N_TRIALS)

In [ ]:
study.best_params

In [ ]:
study.best_value